In [1]:
'''
This notebook demonstrates how to set up model monitoring using SageMaker Model Monitor
Although in real life we would capture real inference data for baseline,
Here we will use that 20% validation data
'''

'\nThis notebook demonstrates how to set up model monitoring using SageMaker Model Monitor. (In a production system you would capture real inference data and set up baselining.)\n'

In [2]:
!pip install awswrangler

In [3]:
import boto3
import time
import datetime
import awswrangler as wr
import pandas as pd
import sagemaker
from sagemaker.model_monitor import DataCaptureConfig, ModelMonitor
from sagemaker.predictor import Predictor
from sagemaker.serializers import JSONSerializer
from sagemaker.processing import ScriptProcessor, ProcessingInput, ProcessingOutput
from sagemaker.model_monitor import DataCaptureConfig, ModelMonitor, CronExpressionGenerator
from sagemaker import get_execution_role, image_uris, Session
from sagemaker.s3 import S3Uploader

bucket_name = "arxiv-project-bucket"
role = "arn:aws:iam::221082214706:role/MYLabRole"
region = "us-east-1"

sess = sagemaker.Session(boto_session=boto3.Session(region_name=region))

# Assume the deployed endpoint is  our"arxiv-clustering-endpoint"
endpoint_name = "arxiv-clustering-endpoint"

# Set prefixes for monitoring outputs
prefix = "sagemaker/Custom-ModelMonitor-Example"
data_capture_prefix = f"{prefix}/datacapture"
s3_capture_upload_path = f"s3://{bucket_name}/{data_capture_prefix}"
ground_truth_upload_path = f"s3://{bucket_name}/{prefix}/ground_truth_data/{datetime.datetime.now():%Y-%m-%d-%H-%M-%S}"
reports_prefix = f"{prefix}/reports"
s3_report_path = f"s3://{bucket_name}/{reports_prefix}"

# For monitoring output where our monitoring job will write its reports
monitoring_output_uri = f"s3://{bucket_name}/{prefix}/schedule/output"

print("Data capture path:", s3_capture_upload_path)
print("Monitoring output path:", monitoring_output_uri)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[02/21/25 06:04:01] INFO     Found credentials from IAM Role:                                   ]8;id=233825;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=636323;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


                    INFO     Found credentials from IAM Role:                                   ]8;id=566873;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=938703;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

Data capture path: s3://arxiv-project-bucket/sagemaker/Custom-ModelMonitor-Example/datacapture
Monitoring output path: s3://arxiv-project-bucket/sagemaker/Custom-ModelMonitor-Example/schedule/output


In [4]:
# Double check to see edits
print(sagemaker.__version__)

2.239.3


In [5]:
!pip install --upgrade pip


In [6]:
!pip install --upgrade sagemaker


In [7]:
print(sagemaker.__version__)

2.239.3


In [8]:
'''
In this cell, we sample a representative subset of the processed training data 
without loading all of it at once. This baseline dataset will be used to compute 
unsupervised quality metrics (basically our silhouette score)
'''

# Define S3 folder where processed training data is stored
train_path = f"s3://{bucket_name}/processed/train/"
# Define baseline dataset output path (NOTE: this is a dataset folder, important since we chunked)
baseline_path = f"s3://{bucket_name}/model-monitoring/baseline/arxiv_baseline.parquet"

# List all Parquet files in the training folder
parquet_files = wr.s3.list_objects(path=train_path)
print("Found the following parquet files:")
print(parquet_files)

[02/21/25 06:04:04] INFO     Found credentials from IAM Role:                                   ]8;id=639727;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=939476;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

Found the following parquet files:
['s3://arxiv-project-bucket/processed/train/arxiv_train_chunk_0.parquet', 's3://arxiv-project-bucket/processed/train/arxiv_train_chunk_0.parquet/e41408732a4345918e42cc0098224655.snappy.parquet', 's3://arxiv-project-bucket/processed/train/arxiv_train_chunk_1.parquet', 's3://arxiv-project-bucket/processed/train/arxiv_train_chunk_1.parquet/32766f83f8ae4c8e8e97920dc5f3896b.snappy.parquet', 's3://arxiv-project-bucket/processed/train/arxiv_train_chunk_10.parquet/1550e13a2c3541569be48303db148a69.snappy.parquet', 's3://arxiv-project-bucket/processed/train/arxiv_train_chunk_11.parquet/7dc98e4f34cb4a0fad60602745bc2fc3.snappy.parquet', 's3://arxiv-project-bucket/processed/train/arxiv_train_chunk_12.parquet/711248b7fc004157ac71d43712d6b108.snappy.parquet', 's3://arxiv-project-bucket/processed/train/arxiv_train_chunk_13.parquet/adf58ace0472479194064be95361104c.snappy.parquet', 's3://arxiv-project-bucket/processed/train/arxiv_train_chunk_14.parquet/f9b5f16acb4d4e74

In [9]:
# Sample from each file (i.e. 1000 rows per file) and then sample down to a target of 10,000 rows
# Probably should reduce to like 100
sample_list = []
target_sample_size = 10000
for key in parquet_files:
    try:
        df_temp = wr.s3.read_parquet(path=key)
    except Exception as e:
        print(f"Error reading {key}: {e}")
        continue
    if len(df_temp) > 0:
        n_rows = min(1000, len(df_temp))
        sample_df = df_temp.sample(n=n_rows, random_state=39)
        sample_list.append(sample_df)
        
if sample_list:
    baseline_df = pd.concat(sample_list, ignore_index=True)
    if len(baseline_df) > target_sample_size:
        baseline_df = baseline_df.sample(n=target_sample_size, random_state=39)
    print("Baseline dataset shape after sampling:", baseline_df.shape)
else:
    raise ValueError("No data samples could be extracted from the processed training files.")

# Need to drop problematic columns ("versions") that cause issues with pyarrow
# Likely important to not include as baseline feature in kmeans cluster
if "versions" in baseline_df.columns:
    baseline_df = baseline_df.drop(columns=["versions"])
    print("Dropped 'versions' column from baseline dataset.")

# Write the baseline dataset to S3 as a dataset (folder of Parquet files)
wr.s3.to_parquet(
    df=baseline_df,
    path=baseline_path,
    dataset=True,
    mode="overwrite",
    compression="snappy"
)
print("Baseline dataset created and uploaded to:", baseline_path)


[02/21/25 06:04:05] INFO     Found credentials from IAM Role:                                   ]8;id=825725;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=210125;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

                    INFO     Skipping checksum validation. Response did not contain one of the  ]8;id=319905;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/httpchecksum.py\httpchecksum.py]8;;\:]8;id=96428;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/httpchecksum.py#481\481]8;;\
                             following algorithms: ['crc32', 'sha1', 'sha256'].                                    

                    INFO     Skipping checksum validation. Response did not contain one of the  ]8;id=709261;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/httpchecksum.py\httpchecksum.py]8;;\:]8;id=528083;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/httpchecksum.py#481\481]8;;\
                             following algorithms: ['crc32', 'sha1', 'sha256'].                                    

                    INFO     Skipping checksum validation. Response did not contain one of the  ]8;id=946777;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/httpchecksum.py\httpchecksum.py]8;;\:]8;id=738795;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/httpchecksum.py#481\481]8;;\
                             following algorithms: ['crc32', 'sha1', 'sha256'].                                    

                    INFO     Skipping checksum validation. Response did not contain one of the  ]8;id=556421;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/httpchecksum.py\httpchecksum.py]8;;\:]8;id=495280;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/httpchecksum.py#481\481]8;;\
                             following algorithms: ['crc32', 'sha1', 'sha256'].                                    

...

Baseline dataset shape after sampling: (10000, 61)
Dropped 'versions' column from baseline dataset.


[02/21/25 06:05:18] INFO     Found credentials from IAM Role:                                   ]8;id=542778;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=377592;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

                    INFO     Found credentials from IAM Role:                                   ]8;id=996433;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=257186;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

                    INFO     Found credentials from IAM Role:                                   ]8;id=685429;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=905023;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

Baseline dataset created and uploaded to: s3://arxiv-project-bucket/model-monitoring/baseline/arxiv_baseline.parquet


In [10]:
# Configure Data Capture for the Endpoint
data_capture_config = DataCaptureConfig(
    enable_capture=True,
    sampling_percentage=100,  # Capture all inference data
    destination_s3_uri=s3_capture_upload_path
)
print("Data capture configuration set.")


[02/21/25 06:05:19] INFO     Found credentials from IAM Role:                                   ]8;id=160484;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=56237;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

Data capture configuration set.


In [11]:
'''
This cell runs a processing job that computes baseline metrics using the custom_baseline.py script.
The custom_baseline.py is expected to read the baseline dataset (folder) and compute metrics (silhouette score).
As a result we need to pass the folder path as input.
'''
baseline_job_name = "arxiv-model-baseline-custom-" + datetime.datetime.now().strftime("%Y%m%d%H%M%S")

script_processor = ScriptProcessor(
    image_uri=image_uris.retrieve("sklearn", region, version="0.23-1", py_version="py3"),
    command=["python3"],
    instance_type="ml.m5.xlarge",
    instance_count=1,
    role=role,
    sagemaker_session=sess,
)

script_processor.run(
    inputs=[
        ProcessingInput(
            source=baseline_path,  
            destination="/opt/ml/processing/input_data",
            s3_data_type="S3Prefix"
        )
    ],
    outputs=[
        ProcessingOutput(
            output_name="baseline_output",
            source="/opt/ml/processing/output"
        )
    ],
    code="custom_baseline.py",
    arguments=[
        "--input_data", "/opt/ml/processing/input_data",
        "--output_dir", "/opt/ml/processing/output",
        "--n_clusters", "5"
    ],
    job_name=baseline_job_name,
    wait=True
)
print("Custom baseline processing job completed.")

                    INFO     Defaulting to only supported image scope: cpu.                       ]8;id=878203;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=79367;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#534\534]8;;\

[02/21/25 06:05:20] INFO     Creating processing-job with name                                      ]8;id=957529;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=790640;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#1575\1575]8;;\
                             arxiv-model-baseline-custom-20250221060519                                            

.............Baseline metrics written to /opt/ml/processing/output/baseline_metrics.json

Custom baseline processing job completed.


In [ ]:
# Below few cells are highly experimental and repeated

In [13]:
'''
This is where we run into issues, creating a Model Monitor Schedule to eval endpoint
For our unsupervised problem, our custom baseline metrics (silhouette score) will serve as our baseline
We should also enable cloudwatch for this. 
'''
monitor = ModelMonitor(
    role=role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
    volume_size_in_gb=20,
    max_runtime_in_seconds=1800,
    sagemaker_session=sess
)

monitor_schedule_name = "arxiv-model-monitor-schedule-" + datetime.datetime.now().strftime("%Y%m%d%H%M%S")

# Create the schedule
# Important note that we no longer use output_s3_uri, but we use "output" as a dictionary
output_config = {
    "MonitoringOutputs": [
         {
             "S3Output": {
                  "S3Uri": monitoring_output_uri,
                  "LocalPath": "/opt/ml/processing/output",
                  "S3UploadMode": "Continuous"
             }
         }
    ]
}

environment = {} # Unsure if we need environment variables

monitor_schedule = monitor.create_monitoring_schedule(
    monitor_schedule_name=monitor_schedule_name,
    endpoint_input=endpoint_name,
    output=output_config,
    schedule_cron_expression=CronExpressionGenerator.hourly(),
)
print("Monitoring schedule created:", monitor_schedule_name)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:31                                                                                   │
│                                                                                                  │
│   28                                                                                             │
│   29 environment = {}  # add any required environment variables if needed                        │
│   30                                                                                             │
│ ❱ 31 monitor_schedule = monitor.create_monitoring_schedule(                                      │
│   32 │   monitor_schedule_name=monitor_schedule_name,                                            │
│   33 │   endpoint_input=endpoint_name,                                                           │
│   34 │   output=output_config,                                                                   │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/model_monitor/model │
│ _monitoring.py:385 in create_monitoring_schedule                                                 │
│                                                                                                  │
│    382 │   │   │   │   endpoint_input=endpoint_input                                             │
│    383 │   │   │   )._to_request_dict()                                                          │
│    384 │   │                                                                                     │
│ ❱  385 │   │   normalized_monitoring_output = self._normalize_monitoring_output_fields(output=o  │
│    386 │   │                                                                                     │
│    387 │   │   statistics_object, constraints_object = self._get_baseline_files(                 │
│    388 │   │   │   statistics=statistics, constraints=constraints, sagemaker_session=self.sagem  │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/model_monitor/model │
│ _monitoring.py:1455 in _normalize_monitoring_output_fields                                       │
│                                                                                                  │
│   1452 │   │                                                                                     │
│   1453 │   │   """                                                                               │
│   1454 │   │   # If the output destination is missing, assign a default destination to it.       │
│ ❱ 1455 │   │   if output.destination is None:                                                    │
│   1456 │   │   │   output.destination = s3.s3_path_join(                                         │
│   1457 │   │   │   │   "s3://",                                                                  │
│   1458 │   │   │   │   self.sagemaker_session.default_bucket(),                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'dict' object has no attribute 'destination'

In [14]:
# Define a dummy output configuration class
class DummyMonitoringOutputConfig:
    def __init__(self, s3_uri, local_path="/opt/ml/processing/output", s3_upload_mode="Continuous"):
        self.destination = s3_uri
        self.local_path = local_path
        self.s3_upload_mode = s3_upload_mode

    def _to_request_dict(self):
        return {
            "MonitoringOutputs": [
                {
                    "S3Output": {
                        "S3Uri": self.destination,
                        "LocalPath": self.local_path,
                        "S3UploadMode": self.s3_upload_mode
                    }
                }
            ]
        }

# Create an instance of the dummy output configuration
dummy_output_config = DummyMonitoringOutputConfig(monitoring_output_uri)

# Create an EndpointInput object for your endpoint
from sagemaker.model_monitor import EndpointInput

endpoint_input = EndpointInput(
    endpoint_name=endpoint_name,
    destination="/opt/ml/processing/input/endpoint"
)

# Create the monitoring schedule
monitor_schedule = monitor.create_monitoring_schedule(
    monitor_schedule_name=monitor_schedule_name,
    endpoint_input=endpoint_input,
    output=dummy_output_config,
    schedule_cron_expression=CronExpressionGenerator.hourly()
)
print("Monitoring schedule created:", monitor_schedule_name)


[02/21/25 06:19:39] WARNING  It seems that this object was already used to create an Amazon ]8;id=786726;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/model_monitor/model_monitoring.py\model_monitoring.py]8;;\:]8;id=680250;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/model_monitor/model_monitoring.py#353\353]8;;\
                             Model Monitoring Schedule. To create another, first delete the                        
                             existing one using my_monitor.delete_monitoring_schedule().                           

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:33                                                                                   │
│                                                                                                  │
│   30 )                                                                                           │
│   31                                                                                             │
│   32 # Create the monitoring schedule                                                            │
│ ❱ 33 monitor_schedule = monitor.create_monitoring_schedule(                                      │
│   34 │   monitor_schedule_name=monitor_schedule_name,                                            │
│   35 │   endpoint_input=endpoint_input,                                                          │
│   36 │   output=dummy_output_config,                                                             │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/model_monitor/model │
│ _monitoring.py:354 in create_monitoring_schedule                                                 │
│                                                                                                  │
│    351 │   │   │   │   "using my_monitor.delete_monitoring_schedule()."                          │
│    352 │   │   │   )                                                                             │
│    353 │   │   │   logger.warning(message)                                                       │
│ ❱  354 │   │   │   raise ValueError(message)                                                     │
│    355 │   │                                                                                     │
│    356 │   │   if not output:                                                                    │
│    357 │   │   │   raise ValueError("output can not be None.")                                   │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: It seems that this object was already used to create an Amazon Model Monitoring Schedule. To create 
another, first delete the existing one using my_monitor.delete_monitoring_schedule().

In [ ]:
# Check Monitoring Execution Status
print("Waiting for the first monitoring execution...")
time.sleep(60)  # Wait one minute

executions = monitor.list_executions()
if executions:
    latest_execution = executions[-1]
    print("Latest monitoring execution details:")
    print(latest_execution.describe())
else:
    print("No monitoring executions found yet.")
